In [1]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from torch.utils.data import Dataset, DataLoader

### preproccessing the data

In [2]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        return image, label

In [ ]:
def prepare_the_data(data:pd.DataFrame):
    # Extract the pixel data and labels
    pixel_columns = [col for col in df.columns if col.startswith('pixel')]
    pixels = data[pixel_columns].values
    labels = data['label'].values

    # Reshape the pixel data to match the image dimensions (3, 32, 32)
    pixels = pixels.reshape(-1, 3, 32, 32)

    # Convert pixel data to float32 and normalize to the range [0, 1]
    pixels = pixels.astype(np.float32) / 255.0

    # Convert NumPy arrays to PyTorch tensors
    pixel_tensors = torch.tensor(pixels)
    label_tensors = torch.tensor(labels)
    
    #normalize
    normalize_transform = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    pixel_tensors = normalize_transform(pixel_tensors)
    
    # Create a CustomDataset instance
    dataset = CustomDataset(pixel_tensors, label_tensors)

    # Create a DataLoader
    batch_size = 128
    train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    
    return train_loader

### Resnet18 model - Tensorboard

In [ ]:
import torchvision.models as models
import torch.nn as nn
from tensorboardX import SummaryWriter

In [ ]:
def resnet18_model_with_tensorboard(num_epochs:int,train_loader):
    # Define the ResNet18 model
    model = models.resnet18(pretrained=False)
    num_classes = 10  # Assuming you have 10 classes in your dataset
    model.fc = nn.Linear(512, num_classes)  # Replace the fully connected layer for the new number of classes

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

    # Set up TensorBoard writer
    writer = SummaryWriter('logs')
    
    #num_epochs = 10  # You can adjust this as needed

    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        total_loss = 0.0

        for i, (images, labels) in enumerate(train_loader):
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        # Average training loss for the epoch
        avg_loss = total_loss / len(train_loader)

        # Write training loss to TensorBoard
        writer.add_scalar('Loss/train', avg_loss, epoch)

    # Close the TensorBoard writer
    writer.close()

In [ ]:
train_loader = prepare_the_data(data...)
resnet18_model_with_tensorboard(30,train_loader)

In [ ]:
tensorboard --logdir=logs